*Analytical Information Systems*

# Tutorial 5 - Predictive Modeling

Matthias Griebel<br>
Lehrstuhl für Wirtschaftsinformatik und Informationsmanagement

SS 2019

<h1>Agenda<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Follow-up-on-data-visualization:-Esquisse" data-toc-modified-id="Follow-up-on-data-visualization:-Esquisse-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Follow up on data visualization: Esquisse</a></span></li><li><span><a href="#Data-Mining" data-toc-modified-id="Data-Mining-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Mining</a></span></li><li><span><a href="#Data-Preparation" data-toc-modified-id="Data-Preparation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data Preparation</a></span><ul class="toc-item"><li><span><a href="#Training,-validation,-and-test-sets" data-toc-modified-id="Training,-validation,-and-test-sets-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Training, validation, and test sets</a></span></li><li><span><a href="#Data-Preparation" data-toc-modified-id="Data-Preparation-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Data Preparation</a></span></li></ul></li><li><span><a href="#Modeling" data-toc-modified-id="Modeling-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Modeling</a></span><ul class="toc-item"><li><span><a href="#Metrics-for-classification" data-toc-modified-id="Metrics-for-classification-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Metrics for classification</a></span></li></ul></li><li><span><a href="#Up-to-you:--Titanic-Passenger-Survival-Classification" data-toc-modified-id="Up-to-you:--Titanic-Passenger-Survival-Classification-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Up to you:  Titanic Passenger Survival Classification</a></span></li></ul></div>

## Follow up on data visualization: Esquisse

The [Esquisse](https://github.com/dreamRs/esquisse) addin allows you to interactively explore your data by visualizing it with the ggplot2 package. It allows you to draw bar plots, curves, scatter plots, histograms, boxplot and sf objects, then export the graph or retrieve the code to reproduce the graph.

<img src="https://github.com/dreamRs/esquisse/raw/master/man/figures/esquisse.gif" style="heigth:50%">

__Esquisse works best in RStudio__

1. Open RStudio
    - For notebooks running on *binderhub*: Copy the first part of the URL from this notebook, e.g.: 
            `https://hub.gke.mybinder.org/user/matjesg-ais_2019-qgp8pz15/` 
    - and paste '/rstudio' at the end
            `https://hub.gke.mybinder.org/user/matjesg-ais_2019-qgp8pz15/rstudio` 
- Start Esquisse
    - Choose `Addins`-> `'ggplot2' builder`
    - Or run `esquisse:::esquisser(viewer = "pane")`


## Data Mining

__CRISP-DM__
<img align="right" src="http://statistik-dresden.de/wp-content/uploads/2012/04/CRISP-DM_Process_Diagram1.png" style="width:50%">

Today, we will focus on 
- `Data Preparation`
- `Modelling`



__Tidymodels__

Similar to its sister package `tidyverse`, it can be used to install and load `tidyverse` packages related to modeling and analysis. Currently, it installs and attaches `dplyr`,`ggplot2`, `purrr` as well as

- `rsample`: Create and summarize different types of resampling objects (e.g. bootstrap, cross-validation)
- `broom`: Summarizes key information about statistical objects in tidy tibbles
- `recipes`: Define a recipe or blueprint that can be used to sequentially define the encodings and preprocessing of the data
- `infer`: tidyverse-friendly statistical inference
- `yardstick`: Tidy methods for measuring model performance


The declared goal of the tidymodels metapackage is to provide a unified modelling synthax similar to scikit-learn in the python domain.

In [ ]:
library(tidymodels)

## Data Preparation

### Training, validation, and test sets


__Information leaks__

Every time you tune a hyperparameter of your model based on the model’s performance on the validation set, some information about the validation data leaks into the model.

__How to avoid overfitting?__


__Simple Hold-Out validation__

- Set apart some fraction of your data as your test set 
- Train on the remaining data, and evaluate on the validation set (part of your the remaining data).
- Once your model is ready for prime time, you test it one final time on the test data.

<img align="center" src="https://elitedatascience.com/wp-content/uploads/2017/06/Train-Test-Split-Diagram.jpg" style="width:70%">

Source: [EliteDataScience](https://elitedatascience.com/model-training)

__K-fold cross-validation and K-fold stratified cross-validation__

- Set apart some fraction of your data as your test set 
- Split the remaining data into `K` partitions of equal size. For each partition `i`, train a model on the remaining `K-1` partitions, and evaluate it on partition `i`.
- Stratified cross-validation splits the data such that the proportions between classes are the same in each fold as they are in the whole dataset 

<img align="center" src="images/05/cv_comparison.png" style="width:90%">

__Resampling with rsample__

<img align="right" src="https://tidymodels.github.io/rsample/rsample_hex_thumb.png" style="width:20%">

`rsample` contains a set of functions that can create different types of resamples and corresponding classes for their analysis.

- Traditional resampling techniques for estimating the sampling distribution of a statistic
- Estimating model performance using a holdout set
- Example: 
```
data_split <- initial_split(data)
train_data <- training(data_split)
test_data <- testing(data_split)
```



___Example: Credit data___

The data set contains anonymized information about the credit status of bank customers.
From https://cran.r-project.org/web/packages/recipes/vignettes/Simple_Example.html

In [ ]:
data("credit_data")
glimpse(credit_data)
?credit_data

___Credit data: Train-Test-Split___

In [ ]:
train_test_split <- initial_split(credit_data, prop = 0.7)
credit_train <- training(train_test_split)
credit_test <- testing(train_test_split)

Is this an appropriate way to split the data?

In [ ]:
sum(credit_data$Status=="bad")/sum(credit_data$Status=="good")
sum(credit_train$Status=="bad")/sum(credit_train$Status=="good")
sum(credit_test$Status=="bad")/sum(credit_test$Status=="good")

___Credit data: Stratified Split___

In [ ]:
set.seed(0)
train_test_split <- initial_split(credit_data, prop = 0.7, strata = "Status")
credit_train <- training(train_test_split)
credit_test <- testing(train_test_split)

In [ ]:
sum(credit_data$Status=="bad")/sum(credit_data$Status=="good")
sum(credit_train$Status=="bad")/sum(credit_train$Status=="good")
sum(credit_test$Status=="bad")/sum(credit_test$Status=="good")

### Data Preparation

<img align="center" src="images/05/data_prep.png" style="width:80%">


__Feature Engeneering__

> "Coming up with features is difficult, time-consuming, requires expert knowledge. Applied machine learning is basically feature engineering." (Andrew Ng)


Putting in plain vanilla variables into our model is often not the best idea (however, often a good start)
- Think about creating new variables
    - combine distance and time to create a speed variable
- Group together similar concepts to reduce factor levels
    - combine “lawyer“ and “judge” to “legal profession”
- Identify thresholds through explorative data analysis and visualization
- Reduce dimensionality of data through PCA or polynomial fitting

__Data Preprocessing with recipes__

<img align="right" src="https://github.com/tidymodels/recipes/raw/master/man/figures/logo.png" style="width:20%">

The idea of the recipes package is to define a recipe or blueprint that can be used to sequentially define the encodings and preprocessing of the data (i.e. feature engineering)

1. __Recipe:__ Define roles (outcome, predictor, etc.) and the steps to be applied to a data set in order to get it ready for data analysis

2. __Prepare:__ Estimate the required parameters from a training set that can be later applied to other data sets

3. __Bake:__ Apply the recipes to the targeted dataset.

___Credit data: Preprocessing___

There are some missing values in these data

In [ ]:
vapply(credit_train, function(x) mean(!is.na(x)), numeric(1))

Rather than remove these, their values will be imputed.

The idea is that the preprocessing operations will all be created using the training set and then these steps will be applied to both the training and test set.

___Credit data: Initial Recipe___

First, we will create a recipe object from the original data and then specify the processing steps.

Recipes can be created manually by sequentially adding roles to variables in a data set.

If the analysis only required outcomes and predictors, the easiest way to create the initial recipe is to use the standard formula method:

In [ ]:
rec_obj <- recipe(Status ~ ., data = credit_train)
rec_obj

The data contained in the data argument need not be the training set; this data is only used to catalog the names of the variables and their types (e.g. numeric, etc.).

___Credit data: Imputation of missing values___

Here, K-nearest neighbor imputation will be used. This works for both numeric and non-numeric predictors and defaults K to five To do this, it selects all predictors then removes those that are numeric:

In [ ]:
imputed <- rec_obj %>%
        step_knnimpute(all_predictors()) 
imputed

___Credit data: Dummy variables___

Since some predictors are categorical in nature (i.e. nominal), it would make sense to convert these factor predictors into numeric dummy variables using step_dummy. To do this, the step selects all predictors then removes those that are numeric.

In [ ]:
ind_vars <- imputed %>%
        step_dummy(all_predictors(), -all_numeric()) 
ind_vars

___Credit data: Standardize___

At this point in the recipe, all of the predictor should be encoded as numeric, we can further add more steps to center and scale them:

In [ ]:
standardized <- ind_vars %>%
        step_center(all_predictors())  %>%
        step_scale(all_predictors()) 
standardized

___Credit data: Prepare___

We can now estimate the means and standard deviations from the training set. The prep function is used with a recipe and a data set:

In [ ]:
trained_rec <- prep(standardized, training = credit_train)
trained_rec

___Credit data: Bake___

Now that the statistics have been estimated, the preprocessing can be applied to the training and test set:

In [ ]:
train_data <- bake(trained_rec, new_data = credit_train) %>% na.omit()
test_data  <- bake(trained_rec, new_data = credit_test)%>% na.omit()

## Modeling

__The modeling phase__
- Various modeling techniques are selected and applied
- Often several techniques for the same problem
- Besides the algorithms, we also need to specify evaluation procedure and metric
- Specific requirements necessitate going back to preparation


__Model Selection & Generalization__

- Learning is an ill-posed problem: data is not sufficient to find a unique solution
    - Supervised learning seeks to identify models that are able to make accurate predictions on unseen data that has similar characteristics as the data set used for training the model
    - This is referred to as generalization and should guide model selection

- To be able generalize a model needs to weigh complexity against observed error:
    - More complex models have lower observed error on training data, might have higher true error (on unknown data)
    - This is related to the bias-variance trade-off



__The Bias-Variance Trade-off__


<img src="http://scott.fortmann-roe.com/docs/docs/BiasVariance/biasvariance.png" style="width:70%">
 
Source: http://scott.fortmann-roe.com/docs/docs/BiasVariance/biasvariance.png

__Modeling with parsnip__

The `parsnip` package is a unified interface to models. This should significantly reduce the amount of syntactical minutia that you’ll need to memorize by having one standardized model function across different packages and by harmonizing the parameter names across models.

In [ ]:
library(parsnip)

___Credit data: Model specification___

To use `parsnip`, you start with a model specification. This is a simple object that defines the intent of the model. We will be using a logistic regression for classifying the credit status:

In [ ]:
credit_model <- logistic_reg()
credit_model

___Credit data: Model engine___

`parsnip` offers a variety of methods to fit this general model. We differentiate these cases by the computational engines, which is a combination of the 
- Estimation type, such as least squares, and 
- Implemention 
    - R package or some other computing platform like Spark or Tensorflow
    - `glm` is used to fit generalized linear models

In [ ]:
credit_model %>%
    set_engine("glm") -> glm_credit_model
glm_credit_model

___Credit data: Fit model___

Here are no additional arguments that we should specify here, so let’s jump to fitting the actual model.

In [ ]:
glm_credit_model %>%
  fit(Status ~ ., data = train_data) -> glm_fit
glm_fit

__Assess Performance with yardstick__

`yardstick` is a package to estimate how well models are working using tidy data principle

In [ ]:
predicted = predict(glm_fit, test_data)
predicted_probs = predict(glm_fit, test_data, type = "prob")

test_data %>%
    select(Status) %>%
    cbind(predicted, predicted_probs) -> res
res

__Confusion Matrix__

In classification problems, the primary source for evaluation metrics is the confusion matrix.

<img src="https://cdn-images-1.medium.com/max/1600/1*-BkpqhN-5fPicMifDQ0SwA.png" style="width:40%">


see https://en.wikipedia.org/wiki/Confusion_matrix

In [ ]:
res %>%
    conf_mat(Status, .pred_class)

### Metrics for classification

__Accuracy__

The accuracy is the proportion of correct classifications. Accuracy is not a reliable metric for the real performance of a classifier, because it will yield misleading results if the data set is unbalanced (that is, when the numbers of observations in different classes vary greatly).

$ACC = \frac{TP+TN}{TP+FP+FN+TN}$

In [ ]:
res %>%
    metrics(Status, .pred_class)

__Precision__

Precision (also called positive predictive value) is the fraction of relevant instances among the retrieved instances.

$Precision = \frac{TP}{TP+FP}$

In [ ]:
res %>%
    precision(Status, .pred_class)

__Recall__

Recall (also known as sensitivity) is the fraction of relevant instances that have been retrieved over the total amount of relevant instances.

$Recall = \frac{TP}{TP+FN}$

In [ ]:
res %>%
    recall(Status, .pred_class)

__ROC Curve__

A receiver operating characteristic curve, or ROC curve, is a graphical plot that illustrates the diagnostic ability of a binary classifier system as its discrimination threshold is varied.

In [ ]:
res %>%
    roc_curve(Status, .pred_bad) %>%
    autoplot()

## Up to you:  Titanic Passenger Survival Classification



__Titanic Passenger Survival Data Set__

This data set provides information on the fate of passengers on the fatal maiden voyage of the ocean liner "Titanic", summarized according to 
- economic status (class)
- sex
- age 
- survival 

The data sets are the individual non-aggregated observations and formatted in a machine learning context with a training sample, a testing sample, and two additional data sets that can be used for deeper machine learning analysis.

In [ ]:
glimpse(titanic::titanic_train)

__Up to you: Titanic Passenger Survival__

Build a classification model that predicts the survival or death of a passenger on the titanic, depending on the ticket class, age, sex, and port of embarkation.

1. Split the data into train and test set

__Up to you: Titanic Passenger Survival__

2. Prepare a recipe for data preprocessing (use ticket class, age, sex, and port of embarkation as variables)

__Up to you: Titanic Passenger Survival__

3. Bake train and test set

__Up to you: Titanic Passenger Survival__

3. Fit the a logistic classification model (`glm`) on the train set

__Up to you: Titanic Passenger Survival__

4. Evaluate the model on the test set
